In [1]:
import torch 
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:3]

['emma', 'olivia', 'ava']

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [ ]:
block_size = 3
X, Y = [], []

for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [173]:
def build_dataset(words):
    block_size = 5
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random 
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182512, 5]) torch.Size([182512])
torch.Size([22860, 5]) torch.Size([22860])
torch.Size([22774, 5]) torch.Size([22774])


In [174]:
C = torch.randn((27, 2))

In [175]:
C[5]

tensor([-0.5429,  2.7619])

In [176]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [177]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [178]:
h = torch.tanh((emb.view(-1, 6) @ W1) + b1)

In [179]:
h

tensor([[-0.9999,  0.9735,  0.9995,  ..., -0.3137,  0.9258, -1.0000],
        [-1.0000,  0.9452,  0.9992,  ...,  0.9999, -0.7606, -1.0000],
        [-0.9999, -0.9997,  0.9947,  ...,  1.0000, -0.9996, -0.9897],
        ...,
        [ 0.9695, -0.5330, -0.6763,  ..., -0.9598, -0.8776, -0.9971],
        [-0.9425,  0.9788, -0.0967,  ...,  0.3744, -0.6922, -0.9994],
        [ 0.0518, -0.5333, -0.3645,  ..., -0.3432, -0.9067, -0.6486]])

In [180]:
h.shape

torch.Size([228146, 100])

In [181]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [182]:
logits = (h @ W2) + b2

In [183]:
logits.shape

torch.Size([228146, 27])

In [184]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)

In [185]:
prob.shape

torch.Size([228146, 27])

In [186]:
loss = -prob[torch.arange(prob.shape[0]), Y].log().mean()

In [187]:
loss

tensor(18.1133)

In [200]:
block_size = 5
C = torch.randn((27, 10))
dim1 = block_size*C.shape[1]
W1 = torch.randn((dim1, 200))
b1 = torch.randn(200)
W2 = torch.randn((200, 27))
b2 = torch.randn(27)
parameters = [C, W1, W2, b1, b2]

In [201]:
sum(p.nelement() for p in parameters)

15897

In [202]:
for p in parameters:
    p.requires_grad = True


In [203]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [204]:
lri = []
lossi = []
stepi = []

In [209]:
for i in range(200000):
    # making batches
    ix = torch.randint(0, Xtr.shape[0], (32,))
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, dim1) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    # backwards pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    stepi.append(i)
    lossi.append(loss.log10().item())

print(loss.item())

1.9054646492004395


In [210]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, dim1) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.1228, grad_fn=<NllLossBackward0>)

In [211]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, dim1) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1524, grad_fn=<NllLossBackward0>)

In [212]:
# training split, dev/validation split, test split
# 80%, 10%, 10%

In [213]:
block_size = 5
for _ in range(20):
    out = []
    context = [0] * block_size

    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, replacement=True)
        context = context[1:] + [ix]
        out.append(itos[ix.item()])
        if ix == 0:
            print(''.join(out[:-1]))
            out = []
            break 

kotillaya
kiona
miray
yuser
adrian
emian
avai
kohenerayah
adeorane
biney
raniel
scezlee
siiannah
irahara
yyoa
emyar
jokdix
ellie
zevyn
dadeigh
